In [ ]:
# A default setup cell.
# It imports environment variables, define 'devtools.debug" as a buildins, set PYTHONPATH, and code auto-reload
# Copy it in other Notebooks


from dotenv import load_dotenv
from rich import print

load_dotenv(verbose=True)

%load_ext autoreload
%autoreload 2
%reset -f


In [ ]:
import os
import tempfile

import pandas as pd

from smolagents import (
    CodeAgent,
    LiteLLMModel,
    tool,
)

from src.ai_core.llm import LlmFactory
from src.utils.config_mngr import global_config
from src.webapp.ui_components.llm_config import llm_config_widget

In [ ]:
from src.utils.singleton import once


MODEL_ID = None
model_name = LlmFactory(llm_id=MODEL_ID).get_litellm_model_name()
llm = LiteLLMModel(model_id=model_name)
print(model_name)


#@st.cache_resource(show_spinner="Load data files")
@once
def get_data() -> pd.DataFrame:
    data_file = global_config().get_path("datasets_root") / "carbon-monitor-data 1.xlsx"
    assert data_file.exists()

    if data_file.name.endswith(".csv"):
        df = pd.read_csv(data_file, decimal=",")
    else:
        df = pd.read_excel(data_file)
    return df


In [ ]:
#@st.cache_resource(show_spinner="Load data files")
@once
def get_data() -> pd.DataFrame:
    data_file = global_config().get_path("datasets_root") / "carbon-monitor-data 1.xlsx"
    assert data_file.exists()

    if data_file.name.endswith(".csv"):
        df = pd.read_csv(data_file, decimal=",")
    else:
        df = pd.read_excel(data_file)
    return df

@tool
def get_data_frame()-> pd.DataFrame:
    """Return a data frame with  data related on CO2 emissions"""
    return get_data()


In [ ]:
get_data().head()

In [ ]:
agent = CodeAgent(
    model=llm,
    tools=[get_data_frame],
    additional_authorized_imports=["pandas", "matplotlib.pyplot", "numpy", "json"],
)

agent.run("what was the CO2 emmissions of Brazil for energy generation in 2023")